In [1]:
import pandas as pd
import json
import numpy as np
import jenks
import copy

In [2]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('Kod').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('Kod').to_dict(orient='index')
okr = pd.read_csv('../scratch/okres_meta.csv').set_index('Kod').to_dict(orient='index')

In [3]:
with open('../scratch/okrsky.geojson', encoding='utf-8') as f:
    jsn = json.load(f)

In [4]:
kill = ['fid', 'gml_id', 'Kod', 'Nespravny', 'ObecKod', 'MomcKod', 'Poznamka', 'PlatiOd', 'PlatiDo', 'IdTransakce', 'GlobalniIdNavrhuZmeny']
mcmo_dump = []
for f in jsn['features']:
    f['properties']['Obec'] = ob[f['properties']['ObecKod']]['Nazev']
    f['properties']['Okres'] = okr[ob[f['properties']['ObecKod']]['OkresKod']]['Nazev']
    f['properties']['Momc'] = None
    f['properties']['isMomc'] = False
    f['properties']['okid'] = str(f['properties']['ObecKod']) + '_' + str(f['properties']['Cislo'])
    
    if f['properties']['MomcKod'] is not None:
        f['properties']['Momc'] = momc[f['properties']['MomcKod']]['Nazev']
        n = copy.deepcopy(f)
        n['properties']['isMomc'] = True
        n['properties']['okid'] = str(f['properties']['MomcKod']) + '_' + str(f['properties']['Cislo'])
        
        for k in kill:
            if k in n['properties']:
                del n['properties'][k]
        
        mcmo_dump.append(n)

    for k in kill:
        if k in f['properties']:
            del f['properties'][k]
            
jsn['features'].extend(mcmo_dump)

In [5]:
okrs = pd.read_csv('../scratch/okrsky_cisla.csv')

In [6]:
okrs.columns

Index(['CIS_OBEC', 'CIS_OBVOD', 'CIS_OKRSEK', 'DATUM_CAS_ZPRAC',
       'ODEVZDANE_OBALKY', 'OPAKOVANE', 'PLATNE_HLASY', 'PORADI_ZPRAC',
       'VYDANE_OBALKY', 'ZAPSANI_VOLICI', 'result'],
      dtype='object')

In [7]:
okrs['okid'] = okrs.CIS_OBEC.astype('str') + '_' + okrs.CIS_OKRSEK.astype('str')

In [8]:
cls = ['CIS_OBEC', 'CIS_OKRSEK', 'DATUM_CAS_ZPRAC', 'OPAKOVANE', 'PORADI_ZPRAC']
okrs.drop(cls, inplace=True, axis=1)

In [9]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [10]:
for fe in jsn['features']:
    if fe['properties']['okid'] in okrs:
        fe['properties'].update(okrs[fe['properties']['okid']])
    else:
        pass
        #print(fe['properties']['okid'])

In [11]:
jsn['features'] = list(filter(lambda x: 'ZAPSANI_VOLICI' in x['properties'], jsn['features']))

In [12]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))